<a href="https://colab.research.google.com/github/ajasja/RosettaCrashCourse/blob/main/notebooks/W02_parametric_design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install pyrosetta 
#@markdown Run cells by clicking on the little ▶ play icon to the left of each cell or by going to Runtime->Run all. 

#@markdown Enter the user name and password obtained from https://els2.comotion.uw.edu/product/pyrosetta

#@markdown The notebook must be connected to Google Drive. 

#@markdown Author: Ajasja Ljubetič (ajasja.ljubetic@gmail.com), inspired by https://nbviewer.org/github/RosettaCommons/PyRosetta.notebooks


#only needed for colabfold
!pip install pyrosettacolabsetup py3dmol https://github.com/RosettaCommons/pyrosetta_viewer3d/archive/ajasja/bundle.zip --quiet
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()



In [ ]:
#@title Helper functions
def view_single(pose):
  import py3Dmol
  import pyrosetta.distributed.io as io
  view = py3Dmol.view(width=500, height=500)
  view.addModel(io.to_pdbstring(pose),'pdb')
  view.setStyle({'model': 0}, {"cartoon": True})
  view.addStyle({'model': 0}, {"stick": True})
  view.zoomTo()
  view.show()
  return None

def compare_poses(pose1, pose2):
  import py3Dmol
  import pyrosetta.distributed.io as io
  view = py3Dmol.view(width=500, height=500)
  view.addModel(io.to_pdbstring(pose1),'pdb')
  view.addModel(io.to_pdbstring(pose2),'pdb')
  view.setStyle({'model': 0}, {"cartoon": {'color': 'orange'}})
  view.addStyle({'model': 0}, {"stick": {'color': 'orange'}})
  view.setStyle({'model': 1}, {"cartoon": {'color': 'blue'}})
  view.addStyle({'model': 1}, {"stick": {'color': 'blue'}})
  view.zoomTo()
  view.show()
  return None  

  def get_multiple_poses(pose, mover):
  """Retunrns multiples poses from a multiple pose mover"""
  poses = [pose]
  for x in range(0,100):
    extra_pose = mover.get_additional_output()
    if extra_pose is None:
        break
    poses.append(extra_pose)   
  return poses

In [ ]:
import pyrosetta; # import the pyrosetta package to access functionality
import pyrosetta.distributed.io as io
pyrosetta.init("-corrections::beta_nov16") # must be called before any other pyrosetta functions. Can accept command line flags
import viewer3d

# Interactive helix bundle builder
Rosetta is capable of building coiled-coil backbones. With the magic of Google colab, ipywidgets and PyRosetta even non-experts can interactively play with backbone design. 


## Bundle parameters ([MakeBundleMover](https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/movers_pages/MakeBundleMover))
* **lenght**: lenght of the helix in residues.
* **r0**: Major helix radius, in Angstroms.
* **omega0**: Major helix twist per residue (in degrees).
* **delta_omega1**: Rotation of a helix about its own axis (in degrees).
* **invert**: run the helix in the other direction (up vs down).
* **z0_offset**: Offset along the global z-axis, in Angstroms.
* **z1_offset**: Offset along the superhelical path through space, in Angstroms.
W1

In [ ]:
v = viewer3d.presets.makeBundle(aa="VAL", num_helices=4, backend="py3Dmol", window_size=(600,600))
v.show()

## TASK 1: Create 5-10 structures by hand
Use the interactive viewer to save 5-10 structures by hand. Save the files by clicking on the save PDB button. Make sure to downlaod the files to your computer!

Hints:
- Use 4 helices.
- Set `omega0` to one of common values (-2.85, 0 or 1.82)
- `R0` should be between 6 and 7 A.
- Keep enough core (orange residues)  

## TASK 2: Use the MakeBundleMover (from XML) to programatically generate a bundle

MakeBundleMover is and example of a mover that is much more convinient to call from XML. Use `XmlObjects.static_get_mover` in `rosetta.protocols.rosetta_scripts` to initate the the mover and programatically use it to generate a bundle. For example write a function that takes a few CC parameters and saves them to a PDB.

Hint: Use documentation for make bundle mover [here](https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/movers_pages/MakeBundleMover). You have to set `delta_omega0` for each helix in the subtag.

In [ ]:
### BEGIN SOLUTION
from rosetta.protocols.rosetta_scripts import XmlObjects
pose=pyrosetta.rosetta.core.pose.Pose()
bundle_maker = XmlObjects.static_get_mover('''
<MakeBundle name="bundle_maker"
                     use_degrees="1"
                     helix_length="28"
                     r0="6.5"
                     omega0="-2.85"
                     delta_t="0"
                     residue_name="ALA">

  <Helix /> #A 
  <Helix delta_omega0="90"  invert="1" /> #B strand
	<Helix delta_omega0="180" /> #C strand
	<Helix delta_omega0="270" invert="1" /> #D strand
</MakeBundle>''')
bundle_maker.apply(pose)
view_single(pose)
print(pose.sequence())

scorefxn = pyrosetta.create_score_function('beta_nov16')
print(scorefxn(pose))
### END SOLUTION

## TASK 3: Sampling a parameter

Use the function written in TASK 2. Use f-strings to pass the parameters in a foor loop. Hint: use `linspace` form `numpy` module to sample the parameters.

In [ ]:
### BEGIN SOLUTION
import numpy
for i in numpt.linspace(5,7, 6):
  pose=pyrosetta.rosetta.core.pose.Pose()
  string=f'''
  <MOVERS>
    <MakeBundle name="design_l"
                     helix_length='{i}'
                     use_degrees="1"
                     r0="5"
                     omega0="0.00"
                     delta_t="0">

  <Helix /> #A 
  <Helix delta_omega0="90"  invert="1" /> #B strand
	<Helix delta_omega0="180" /> #C strand
	<Helix delta_omega0="270" invert="1" /> #D strand
</MakeBundle>
    </MOVERS>'''
  print(string)
  test_mover= XmlObjects.create_from_string(string).get_mover('design_l')
  test_mover.apply(pose)
  #view_single(pose) 
### END SOLUTION

## TASK 4: Choose 5-10 pre-screend backbones

To be on the safe side that you will get at least some backbones that can be sucessfully looped and designed, choose 5-10 structures from a [set of pre-screend scaffolds](https://drive.google.com/file/d/1eV1gcWbTIXIsBg39asC7y40rA26hDVsp/view?usp=sharing). These scaffolds have previusly been designsed, so they should be designable. 


# Loop design
There are several methods to design loops. Desing of longer loops is possible but has low sucess rates. Here we will use a fragment-based method whihc is best for shoort loops (up to 5 residues). A database base of loops clustered from the pdb is needed.


In [ ]:
#@title install database of loops
!pip install gdown -q
#ss_grouped_vall_helix_shortLoop.h5
!gdown 1k0iNO846dl8_KVZ92OEcRyt1MvS8-RL2 
#bundle-up-down.pdb
!gdown 11NLjlRY2mUCa4lt1wplVPMy424KmNV9f

In [ ]:
import pyrosetta; # import the pyrosetta package to access functionality
import pyrosetta.distributed.io as io
# fragment store must be passed as command line argument
# the default sidechain on loops will be ALA
# If you have multiple pyrosetta init calls, sometimes things might not work as expected. Runtime -> Restart Runtime would be a clean solution, but it does not seem necessary in this case.
pyrosetta.init("-corrections::beta_nov16 -indexed_structure_store:fragment_store /content/ss_grouped_vall_helix_shortLoop.h5 -remodel::generic_aa ALA") 


## Near native loop closer
Connects two chains with separate chain IDs.
This is a **Multiple Pose Mover** so it returns an set of poses.

Loops returned are sorted in order of ascending RMSD of loop and helix.


In [ ]:
# https://new.rosettacommons.org/docs/wiki/scripting_documentation/RosettaScripts/xsd/mover_NearNativeLoopCloser_type
from pyrosetta.rosetta.protocols.rosetta_scripts import XmlObjects

# For production runs use RMSthreshold="0.5"
native_loop = XmlObjects.static_get_mover(f'''<NearNativeLoopCloser  name="make_loop"  
loopLengthRange="2,5"  RMSthreshold="1"
chainBeforeLoop="A" chainAfterLoop="B"
chain="A"
output_all="1" max_number_of_results="20"
resAdjustmentRangeSide1="-3,0" resAdjustmentRangeSide2="-3,0" 
allowed_loop_abegos="AGBA,ABBA,AGBBA,ABABA,ABBBA,AGABBA,ABBBBA,AGBBBA" 
/>''')

pose = pyrosetta.io.pose_from_pdb('bundle-up-down.pdb')
# Takes about 45 s per loop
native_loop.apply(pose)

poses = get_multiple_poses(pose, native_loop)
  
import viewer3d
view = viewer3d.init(poses, window_size=(600,600))+ viewer3d.setStyle()

view.show()

## Connect chains mover
One option would be to manually make each loop. There is also a convenience mover: **`ConnectChainsMover`**. 

The mover loops all chains according to a specification: 

- [A+B+C+D] -- makes a single chain bundle
- [A+B, C+D] -- make a two-chain heterodimer

For each loop it chooses the one with the smallest RMSD, **so only one pose** is returned. 


In [ ]:

# https://new.rosettacommons.org/docs/wiki/scripting_documentation/RosettaScripts/xsd/mover_ConnectChainsMover_type

# Adjust chain connections for other problems
connect_chains = XmlObjects.static_get_mover(f'''<ConnectChainsMover name="make_loops"  
loopLengthRange="2,5" 
chain_connections="[A+B+C+D]"
resAdjustmentRangeSide1="-3,0" resAdjustmentRangeSide2="-3,0" 
allowed_loop_abegos="AGBA,ABBA,AGBBA,ABABA,ABBBA,AGABBA,ABBBBA,AGBBBA" 
 />''')

#This will take about 3 min
pose = pyrosetta.io.pose_from_pdb('bundle-up-down.pdb')
connect_chains.apply(pose)
pose.dump_pdb('bundle-up-down-relooped.pdb')

In [ ]:
pose1=pyrosetta.pose_from_file('bundle-up-down-relooped.pdb')
pose2=pyrosetta.pose_from_file('bundle-up-down.pdb')
compare_poses(pose1, pose2)

## Task: Reloop 10 of your choosen bundles
Use the `ConnectChainsMover` to reloop your designs.

Use a for loop to load the pdbs and run the mover for each pose.

Hint:

Use the following code to loop over pdb files:

from glob import glob
for a_file in glob('*.pdb'):
  print(a_file)


In [ ]:
# get a list of PDBs
from glob import glob
pdbs = glob('*.pdb')

In [ ]:
#This will take 30 min to run
### BEGIN SOLUTION
for pdb in pdbs:
  print(pdb)
  pose = pyrosetta.pose_from_pdb(pdb)
  try:
    connect_chains.apply(pose)
    pose.dump_pdb('looped_'+pdb)
  except: 
    print("Could not reloop "+pdb)
### END SOLUTION